In [52]:
import pandas as pd
import datetime
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [53]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

In [54]:
df.head()

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-01,1.13,1.00,1.00,1.00,1.03,1.10,1.01,1.03,1.00,1.03,...,1.00,1.00,1.00,0.93,1.00,0.99,1.00,1.00,1.00,0.84
1999-12-01,1.11,1.00,1.00,1.00,0.92,1.02,1.10,1.00,1.00,0.97,...,1.00,1.00,1.00,1.18,1.00,1.01,1.00,1.00,1.00,0.93
2000-01-01,1.02,1.00,1.00,1.00,0.96,0.88,0.89,0.73,1.00,1.00,...,1.00,1.00,1.00,0.85,1.00,1.22,1.00,1.00,1.00,0.93
2000-02-01,1.00,1.01,0.81,1.23,1.08,0.96,1.25,1.95,1.18,0.93,...,1.03,0.94,1.07,1.04,1.47,1.97,1.13,0.95,1.01,0.94


### Ticker to index

In [55]:
df_T = df.copy(True)
df_T.rename_axis('ticker', inplace=True)
df_T = df_T.T
df_T.head(5)

ticker,1999-10-01,1999-11-01,1999-12-01,2000-01-01,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01,2000-07-01,...,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01
RS1.L,NaN,1.13,1.11,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,1.11,0.96,0.96,0.96,1.01,0.95,1.00,1.19,0.96,0.99
KE,NaN,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,...,1.09,0.88,0.95,0.96,0.97,1.10,0.96,1.08,0.78,0.95
TEG.DE,NaN,1.00,1.00,1.00,0.81,1.12,1.00,0.90,1.12,1.00,...,1.05,0.99,0.86,1.12,1.06,1.06,0.96,1.02,1.06,1.10
LEG.DE,NaN,1.00,1.00,1.00,1.23,1.03,1.00,1.00,1.00,1.00,...,1.13,0.98,0.88,1.17,1.00,1.02,0.97,1.06,1.08,1.08
SCS,NaN,1.03,0.92,0.96,1.08,0.98,0.99,1.04,1.39,1.01,...,1.10,0.94,1.09,0.95,0.92,1.14,0.95,1.12,0.98,0.93


### Dates as row

In [56]:
df.loc['1999-11-01', 'RS1.L']

1.13

In [57]:
dt = datetime.datetime.strptime('2005-11-01', "%Y-%m-%d")
df.loc[(dt - pd.DateOffset(months=12)).strftime('%Y-%m-%d'), 'SCS']

1.02

In [58]:
def getvalue(data, date, ticker, months):
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    try:
        m = months * -1 if months > 0 else months
        offset = pd.DateOffset(months=m) if months > 0 else - pd.DateOffset(months=months)
        return df.loc[(dt + offset).strftime('%Y-%m-%d'), ticker]
    except KeyError:
        return None

In [59]:
df_tabular = pd.DataFrame(columns=['ticker', 'month', 'year', 'monthly_return'])

tickers = df.columns
dates = df.index

ticker_column = []
month_column = []
year_column = []
date_column = []
monthly_return_column_minus12 = []
monthly_return_column_minus11 = []
monthly_return_column_minus10 = []
monthly_return_column_minus9 = []
monthly_return_column_minus8 = []
monthly_return_column_minus7 = []
monthly_return_column_minus6 = []
monthly_return_column_minus5 = []
monthly_return_column_minus4 = []
monthly_return_column_minus3 = []
monthly_return_column_minus2 = []
monthly_return_column_minus1 = []
monthly_return_column = []
monthly_return_column_plus1 = []

for i, ticker in enumerate(tickers):
    for j, date in enumerate(dates):
        ticker_column.append(ticker)
        #
        dt = datetime.datetime.strptime(date, "%Y-%m-%d")
        month_column.append(dt.month)
        year_column.append(dt.year)
        date_column.append(date)
        #
        monthly_return = df.iloc
        monthly_return_column_minus12.append( getvalue(df, date, ticker, -12) )
        monthly_return_column_minus11.append( getvalue(df, date, ticker, -11) )
        monthly_return_column_minus10.append( getvalue(df, date, ticker, -10) )
        monthly_return_column_minus9.append( getvalue(df, date, ticker, -9) )
        monthly_return_column_minus8.append( getvalue(df, date, ticker, -8) )
        monthly_return_column_minus7.append( getvalue(df, date, ticker, -7) )
        monthly_return_column_minus6.append( getvalue(df, date, ticker, -6) )
        monthly_return_column_minus5.append( getvalue(df, date, ticker, -5) )
        monthly_return_column_minus4.append( getvalue(df, date, ticker, -4) )
        monthly_return_column_minus3.append( getvalue(df, date, ticker, -3) )
        monthly_return_column_minus2.append( getvalue(df, date, ticker, -2) )
        monthly_return_column_minus1.append( getvalue(df, date, ticker, -1) )
        monthly_return_column.append(df.loc[date, ticker])
        monthly_return_column_plus1.append( getvalue(df, date, ticker, 1) )


df_tabular = pd.DataFrame(data={
    'ticker': ticker_column,
    'month': month_column,
    'year': year_column,
    'date': date_column,
    'm_return(t-12)': monthly_return_column_minus12,
    'm_return(t-11)': monthly_return_column_minus11,
    'm_return(t-10)': monthly_return_column_minus10,
    'm_return(t-9)': monthly_return_column_minus9,
    'm_return(t-8)': monthly_return_column_minus8,
    'm_return(t-7)': monthly_return_column_minus7,
    'm_return(t-6)': monthly_return_column_minus6,
    'm_return(t-5)': monthly_return_column_minus5,
    'm_return(t-4)': monthly_return_column_minus4,
    'm_return(t-3)': monthly_return_column_minus3,
    'm_return(t-2)': monthly_return_column_minus2,
    'm_return(t-1)': monthly_return_column_minus1,
    'm_return(t)': monthly_return_column,
    'm_return_target(t+1)': monthly_return_column_plus1
})

In [60]:
df_tabular

,ticker,month,year,date,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),m_return(t-3),m_return(t-2),m_return(t-1),m_return(t),m_return_target(t+1)
0,RS1.L,10,1999,1999-10-01,1.00,1.00,1.0,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13,NaN,NaN
1,RS1.L,11,1999,1999-11-01,1.00,1.00,1.0,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13,NaN
2,RS1.L,12,1999,1999-12-01,1.00,1.00,1.0,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1.13
3,RS1.L,1,2000,2000-01-01,1.01,1.00,1.0,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11
4,RS1.L,2,2000,2000-02-01,1.00,1.01,1.0,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495895,UNP,5,2024,2024-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98,0.96
495896,UNP,6,2024,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98,0.98
495897,UNP,7,2024,2024-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09,0.98
495898,UNP,8,2024,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,1.04,1.09


### Ticker Labeling

In [61]:
le = LabelEncoder()
df_tabular['stock_ticker_label'] = le.fit_transform(df_tabular['ticker'])
df_tabular.drop(columns=['ticker'], inplace=True)

### Train & Test

In [62]:
train, test = df_tabular.loc[df_tabular['year'] < 2021], df_tabular.loc[df_tabular['year'] >= 2021]


#### Remove NaN

In [63]:
train = train.dropna()

#### Input & Target

In [64]:
target_key = 'm_return_target(t+1)'

columns = train.columns.to_list()
columns.remove(target_key)
columns.remove('date') # date column should not be part of training
#
X_train = train[columns]
y_train = train[[target_key]]

X_test = test[columns]
y_test = test[[target_key]]

In [65]:
target_key

'm_return_target(t+1)'

In [66]:
y_test

,m_return_target(t+1)
255,1.09
256,1.01
257,1.11
258,1.02
259,1.07
...,...
495895,0.96
495896,0.98
495897,0.98
495898,1.09


### Save Dataframe

In [67]:
df_tabular.to_csv('../../../data/train_test/df_tabular.csv')

X_train.to_csv('../../../data/train_test/df_tabular_xtrain.csv')
y_train.to_csv('../../../data/train_test/df_tabular_ytrain.csv')
X_test.to_csv('../../../data/train_test/df_tabular_xtest.csv')
y_test.to_csv('../../../data/train_test/df_tabular_ytest.csv')

## Multivariate

In [68]:
df_overview

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,trailing_pe,beta,...,industry_Transportation,industry_Transportation Infrastructure,industry_Utilities,stock_exchange_ETR,stock_exchange_LON,stock_exchange_NAS,stock_exchange_NYS,stock_exchange_TKS,stock_ticker_label,trailing_pe_scaled
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L,3.668791e+09,4.365861e+09,19.858974,0.863,...,0,0,0,0,1,0,0,0,1371,0.001718
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,4.281351e+08,3.938843e+08,21.370369,1.258,...,0,0,0,0,0,1,0,0,1018,0.001849
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE,2.765612e+09,2.765612e+09,60.117207,1.037,...,0,0,0,1,0,0,0,0,1485,0.005203
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE,6.927171e+09,6.927171e+09,60.117207,0.957,...,0,0,0,1,0,0,0,0,1056,0.005203
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.492860e+09,1.373431e+09,17.210526,1.314,...,0,0,0,0,0,0,1,0,1400,0.001489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.127629e+07,9.127629e+07,56.325687,0.594,...,0,0,0,0,0,1,0,0,1040,0.004875
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L,7.018797e+08,7.018797e+08,2035.000000,1.238,...,0,0,0,0,1,0,0,0,1130,0.176153
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.940312e+09,1.940312e+09,50.163414,1.191,...,0,0,0,0,0,1,0,0,1165,0.004341
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T,1.704423e+11,1.704423e+11,27.933996,0.585,...,0,0,0,0,0,0,0,1,141,0.002417


In [69]:
X_train

,month,year,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),m_return(t-3),m_return(t-2),m_return(t-1),m_return(t),stock_ticker_label
2,12,1999,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1.11,1371
3,1,2000,1.01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.02,1371
4,2,2000,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1371
5,3,2000,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1371
6,4,2000,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495850,8,2020,0.99,0.99,0.98,1.01,1.01,1.08,1.04,0.95,1.03,1.15,0.90,1.03,1.11,1549
495851,9,2020,0.91,0.99,0.99,0.98,1.01,1.01,1.08,1.04,0.95,1.03,1.15,0.90,1.03,1549
495852,10,2020,1.23,0.91,0.99,0.99,0.98,1.01,1.01,1.08,1.04,0.95,1.03,1.15,0.90,1549
495853,11,2020,0.98,1.23,0.91,0.99,0.99,0.98,1.01,1.01,1.08,1.04,0.95,1.03,1.15,1549


In [70]:
X_train_multi = X_train.merge(df_overview, on='stock_ticker_label')

In [71]:
X_test_multi = X_test.merge(df_overview, on='stock_ticker_label')

### Drop unwanted columns

In [72]:
 X_train_multi.columns.tolist()

['month',
 'year',
 'm_return(t-12)',
 'm_return(t-11)',
 'm_return(t-10)',
 'm_return(t-9)',
 'm_return(t-8)',
 'm_return(t-7)',
 'm_return(t-6)',
 'm_return(t-5)',
 'm_return(t-4)',
 'm_return(t-3)',
 'm_return(t-2)',
 'm_return(t-1)',
 'm_return(t)',
 'stock_ticker_label',
 'company_name',
 'company_esg_score',
 'company_esg_score_group',
 'industry',
 'stock_exchange',
 'stock_ticker_symbol',
 'market_capital',
 'market_capital_euro',
 'trailing_pe',
 'beta',
 'return_on_equity',
 'earnings_growth',
 'company_esg_score_scale',
 'market_capital_scale',
 'score',
 'return_rate_1y_avg',
 'return_rate_5y_avg',
 'return_rate_10y_avg',
 'return_rate_25y_avg',
 'volatility_1y',
 'volatility_5y',
 'volatility_10y',
 'volatility_25y',
 'industry_Auto Components',
 'industry_Automobiles',
 'industry_Banks',
 'industry_Building Products',
 'industry_Chemicals',
 'industry_Commercial Services',
 'industry_Construction Materials',
 'industry_Consumer Durables',
 'industry_Consumer Services',
 '

In [73]:
X_train_multi[['company_esg_score', 'company_esg_score_group']]

,company_esg_score,company_esg_score_group
0,4.5,Negligible ESG Risk
1,4.5,Negligible ESG Risk
2,4.5,Negligible ESG Risk
3,4.5,Negligible ESG Risk
4,4.5,Negligible ESG Risk
...,...,...
418204,20.0,Low ESG Risk
418205,20.0,Low ESG Risk
418206,20.0,Low ESG Risk
418207,20.0,Low ESG Risk


In [74]:
cols = X_train_multi.columns.tolist()
#
for col in ['return_rate_1y_avg', 'return_rate_5y_avg', 'return_rate_10y_avg', 'return_rate_25y_avg',
            'volatility_1y', 'volatility_5y', 'volatility_10y', 'volatility_25y', 'score',
            'company_name', 'company_esg_score_group', 'stock_ticker_symbol',
            'industry', 'stock_exchange']:
    while col in cols:
        cols.remove(col)
#
X_train_multi = X_train_multi[cols]
X_test_multi = X_test_multi[cols]

In [75]:
X_train_multi

,month,year,m_return(t-12),m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),...,industry_Traders & Distributors,industry_Transportation,industry_Transportation Infrastructure,industry_Utilities,stock_exchange_ETR,stock_exchange_LON,stock_exchange_NAS,stock_exchange_NYS,stock_exchange_TKS,trailing_pe_scaled
0,12,1999,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
1,1,2000,1.01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
2,2,2000,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.01,...,0,0,0,0,0,1,0,0,0,0.001718
3,3,2000,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
4,4,2000,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418204,8,2020,0.99,0.99,0.98,1.01,1.01,1.08,1.04,0.95,...,0,1,0,0,0,0,0,1,0,0.002044
418205,9,2020,0.91,0.99,0.99,0.98,1.01,1.01,1.08,1.04,...,0,1,0,0,0,0,0,1,0,0.002044
418206,10,2020,1.23,0.91,0.99,0.99,0.98,1.01,1.01,1.08,...,0,1,0,0,0,0,0,1,0,0.002044
418207,11,2020,0.98,1.23,0.91,0.99,0.99,0.98,1.01,1.01,...,0,1,0,0,0,0,0,1,0,0.002044


In [76]:
X_train_multi.to_csv('../../../data/train_test/df_tabular_xtrain_multi.csv')
X_test_multi.to_csv('../../../data/train_test/df_tabular_xtest_multi.csv')